In [2]:
import tweepy
from pprint import pprint
import jsonpickle
from tweepy import Stream
from tweepy.streaming import StreamListener
import time
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
auth = tweepy.OAuthHandler('iEk2lB2S9h6qdi9bG2F3OOfVX','LHOn4yNXL5ggl193syNeYaMs2jHGlw7zwDVLFc2pjVWW9XD5LO')
auth.set_access_token('1450808706-r0tIB7jGbOeHbz5EiXkUed2HdnTyNqZFGSZINTL','UCkmKWNSK6EYvk5MTT75GJERl7MPwGOMEXAsjqFnhmlBw')

api = tweepy.API(auth)
if (not api):
    print("Authentication failed :(")
else:
    print("Authentication successfull!!! :D")

Authentication successfull!!! :D


In [4]:
df = pd.read_csv(
    'corpus.txt', 
    delimiter='\t', 
    names=['event_mention_id', 'tweet_id', 'event_instance_id', 'event_mention_trigger', 'timestamp', 'text']
)

In [5]:
df1 = {
    'tweet_id':list(),
    'text':list()
}
tweets = []
dataset = []
tweet_ids = df['tweet_id'].values.tolist()

In [ ]:
for i in tqdm(range(0, len(tweet_ids), 100)):
    statuses = api.statuses_lookup(tweet_ids[i: i + 100])
    dataset.append(statuses)
    
    for idx, status in enumerate(statuses):
        tweet = df.iloc[[idx]]
        df1['tweet_id'].append(status._json['id'])
        df1['text'].append(status._json['text'])
    df_prime = pd.DataFrame(df1)
    df_prime.to_csv('fetched_tweets.csv')
    time.sleep((15 * 60) + 2)

In [6]:
def transform(line):
    line = line.lower()
    line = line.replace("'''", "'")
    line = line.replace("isn't", "is not")
    line = line.replace("wasn't", "was not")
    line = line.replace("didn't", "did not")
    line = line.replace("don't", "do not")
    line = line.replace("dont", "do not")

    line = line.replace("doesn't", "does not")
    line = line.replace("doesnt", "does not")

    line = line.replace("won't", "will not")
    line = line.replace("wont", "will not")

    line = line.replace("wouldn't", "would not")
    line = line.replace("can't", "can not")
    line = line.replace("cannot", "can not")

    line = line.replace("hasn't", "has not")
    line = line.replace("*questioned", "questioned")
    line = line.replace("targets", "target")
    line = line.replace("=", "")
        
    return line


In [7]:
import tokenizer as tk
import re

In [8]:
df1 = pd.DataFrame(df1)
def cleanTweet(tweet):
    s = " ".join(tk.tokenize(tweet[:-1]))
    x = re.sub(r'[^\x00-\xf3]', '', s)
    return x


In [9]:
df_main = {
    'event_mention_id': list(), 
    'tweet_id': list(), 
    'event_instance_id': list(), 
    'event_mention_trigger': list(), 
    'timestamp': list(), 
    'text':list()
}

In [9]:
for tweet_id in df1['tweet_id'].values:
    tweet = df[tweet_id == df['tweet_id']]
    for key in df_main.keys():
        if key != 'text':
            df_main[key].append(tweet[key].values[0])
    df_main['text'].append(df1[df1['tweet_id'] == tweet_id]['text'].values[0])

In [10]:
df_main = pd.DataFrame(df_main)

,event_mention_id,tweet_id,event_instance_id,event_mention_trigger,timestamp,text


In [12]:
df_main['text'] = df_main['text'].apply(transform)
df_main['text'] = df_main['text'].apply(cleanTweet)

In [16]:
df_main.to_csv('generated_dataset.txt',sep='\t')

In [34]:
df_main = pd.read_csv(
    'generated_dataset.txt', 
    sep='\t', 
    names=[
        'event_mention_id', 
        'tweet_id', 
        'event_instance_id', 
        'event_mention_trigger', 
        'timestamp', 
        'text'
    ]
)

In [35]:
for tw in df_main['text'].values:
    if "@po_s" in tw:
        print("YAY")

YAY
YAY
YAY
YAY


# Word2Vec

In [36]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

In [37]:
class Tweet_Reader:
    def __init__(self, df):
        self.dataframe = df

    def __iter__(self):
        for idx in self.dataframe.index:
            temp = self.dataframe['text'][idx]
            temp = tk.tokenize(temp.lower())
            yield temp

In [38]:
path = get_tmpfile("word2vec.model")
tweets = Tweet_Reader (df_main) # a memory-friendly iterator

model = Word2Vec(tweets, size=100, window=5, min_count=1, workers=4, iter=30)
model
model.save("word2vec.model")

In [39]:
from torchtext.data import Field
from tqdm.notebook import tqdm
import torch
from torchtext.vocab import Vocab

## Construct Vocab

In [40]:
v = {}
for word, vocab_obj in model.wv.vocab.items():
    v[word] = vocab_obj.count
v['<unk>'] = 0
v['<pad>'] = 0

In [41]:
vc = Vocab(v, specials=['<pad>'])

In [42]:
word2vec_vectors = []
for token, idx in tqdm(vc.stoi.items()):
    if token in model.wv.vocab.keys():
        word2vec_vectors.append(torch.FloatTensor(model[token]))
    else:
        word2vec_vectors.append(torch.zeros(100))

/home/manan/miniconda3/envs/IRE_Project/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [43]:
vc.set_vectors(vc.stoi, word2vec_vectors, 100)

In [44]:
pre_trained_emb = torch.FloatTensor(vc.vectors)

In [45]:
embedding = torch.nn.Embedding.from_pretrained(pre_trained_emb)

In [46]:
embedding(torch.Tensor([vc.stoi['<pad>']]).long())

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]])

In [47]:
import pickle

In [48]:
with open("vocab.pkl", 'wb') as f:
    pickle.dump(vc, f)